In [85]:
import json, operator, requests, time, pycm, pandas as pd
from langchain import HuggingFaceHub, OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from conceptual_engineering_toolkit import Concept, Entity
from datetime import datetime
from string import Template
from pathlib import Path

In [95]:
DIRECTORY = "caligraph_experiments"

SPARQL_ENDPOINT = "http://caligraph.org/sparql"

QUERY_HEADERS = {
    'User-Agent': 'ConceptualEngineeringAgent/0.2 (https://github.com/bradleypallen/conceptual-engineering-using-llms; b.p.allen@uva.nl)'
}

CLASS_QUERY_LIMIT = 250

E_QUERY_LIMIT = 20

MIN_INSTANCE_COUNT = 100

CLASS_QUERY_TEMPLATE = Template("""SELECT ?class ?label ?superClass (COUNT(?instance) AS ?instanceCount) 
WHERE {
   ?class rdf:type owl:Class ;
        rdfs:label ?label ;
        rdfs:subClassOf ?superClass .
   ?instance rdf:type ?class .
}
GROUP BY ?class ?label ?superClass
HAVING (COUNT(?instance) > $minInstanceCount)
LIMIT $limit
""")

POSITIVES_QUERY_TEMPLATE = Template("""SELECT ?instance ?label
WHERE {
  ?class rdfs:subClassOf* <$class> .
  ?instance rdf:type ?class ;
      rdfs:label ?label .
}
LIMIT $limit
""")

NEGATIVES_QUERY_TEMPLATE = Template("""SELECT ?instance ?label
WHERE {
  ?subclassOfSuperClass rdfs:subClassOf* <$superClass> .
  ?instance rdf:type ?subclassOfSuperClass ;
       rdfs:label ?label .
 
  FILTER NOT EXISTS {
    ?subclassOfClass rdfs:subClassOf* <$class> .
    ?instance rdf:type ?subclassOfClass .
  }
}
LIMIT $limit
""")
                                  
CLASS_DEFINITION_FROM_LABEL_PROMPT_TEMPLATE = Template("""Define the concept "$label". 
Work step by step and check your facts. State your definition in the manner of a dictionary.
""")

CLASS_DEFINITION_FROM_TURTLE_PROMPT_TEMPLATE = Template("""Using the following set of RDF statements, 
define the concept "$label". Work set by step and check your facts. State your definition in the manner 
of a dictionary.
                                                           
Turtle: $turtle'
""")

INSTANCE_DESCRIPTION_PROMPT_TEMPLATE = Template("""Summarize the following set of RDF statements 
describing the entity "$label". Work set by step and check your facts. State your summarization 
in the manner of the first paragraph of an encylopedia article on the topic.
                                                   
Turtle: $turtle'
""")

def classes_for_evaluation():
    query = CLASS_QUERY_TEMPLATE.substitute({"minInstanceCount": MIN_INSTANCE_COUNT, "limit": CLASS_QUERY_LIMIT})
    response = requests.get(SPARQL_ENDPOINT, params={'query' : query, 'format' : 'json'}, headers=QUERY_HEADERS)
    response.raise_for_status()
    return sorted([ 
            { 
                "id": candidate["class"]["value"], 
                "label": candidate["label"]["value"], 
                "superClassId": candidate["superClass"]["value"], 
                "instanceCount": candidate["instanceCount"]["value"] 
            } 
            for candidate in response.json()["results"]["bindings"] 
        ], 
        key=operator.itemgetter("instanceCount"), 
        reverse=True
    )

def positive_examples(cls):
    query = POSITIVES_QUERY_TEMPLATE.substitute({"class": cls["id"], "limit": E_QUERY_LIMIT})
    response = requests.get(SPARQL_ENDPOINT, params={'query' : query, 'format' : 'json'}, headers=QUERY_HEADERS)
    response.raise_for_status()
    return [ 
        { 
            "id": instance["instance"]["value"], 
            "label": instance["label"]["value"] 
        } 
        for instance in response.json()["results"]["bindings"] 
    ]

def negative_examples(cls):
    query = NEGATIVES_QUERY_TEMPLATE.substitute({"class": cls["id"], "superClass": cls["superClassId"], "limit": E_QUERY_LIMIT})
    response = requests.get(SPARQL_ENDPOINT, params={'query' : query, 'format' : 'json'}, headers=QUERY_HEADERS)
    response.raise_for_status()
    return [ 
        { 
            "id": instance["instance"]["value"], 
            "label": instance["label"]["value"] 
        } 
        for instance in response.json()["results"]["bindings"] 
    ]

def turtle(id):
    response = requests.get(SPARQL_ENDPOINT, params={'query' : f'DESCRIBE <{id}>', 'format' : 'text\turtle'}, headers=QUERY_HEADERS)
    response.raise_for_status()
    return response.text

def class_definition_from_label(label):
    return ChatOpenAI(model="gpt-4").predict(CLASS_DEFINITION_FROM_LABEL_PROMPT_TEMPLATE.substitute({"label": label}))

def class_definition_from_turtle(id, label):
    return ChatOpenAI(model="gpt-4").predict(CLASS_DEFINITION_FROM_TURTLE_PROMPT_TEMPLATE.substitute({"label": label, "turtle": turtle(id)}))

def instance_description(id, label):
    return ChatOpenAI(model="gpt-4").predict(INSTANCE_DESCRIPTION_PROMPT_TEMPLATE.substitute({"label": label, "turtle": turtle(id)}))

In [97]:
def evaluate(cls):
    positives = positive_examples(cls)
    negatives = negative_examples(cls)
    concept = Concept(cls["id"], cls["label"], class_definition_from_label(cls["label"]), "gpt-4", 0.1)
    df_positives = pd.DataFrame.from_records(positives)
    df_positives["actual"] = "positive"
    df_negatives = pd.DataFrame.from_records(negatives)
    df_negatives["actual"] = "negative"
    df_data = pd.concat([df_positives, df_negatives], ignore_index=True, axis=0)
    df_data["description"] = df_data.apply(lambda ex: instance_description(ex["id"], ex["label"]), axis=1)
    predictions = [ concept.classify(Entity(ex["id"], ex["label"], ex["description"])) for ex in df_data.to_dict("records") ]
    df_predictions = pd.DataFrame(predictions, columns = [ 'predicted', 'rationale' ])
    df_predictions["predicted"] = df_predictions["predicted"].str.lower()
    df_results = pd.concat([df_data, df_predictions], axis=1)
    cm = pycm.ConfusionMatrix(df_results["actual"].tolist(), df_results["predicted"].tolist(), digit=2, classes=[ 'positive', 'negative' ])
    evaluation = { "created": datetime.now().isoformat(), "concept": concept.to_json(), "data": df_results.to_dict('records'), "confusion_matrix": cm.matrix, }
    experiment_filename = f'{DIRECTORY}/{cls["label"].replace(" ","_")}/{evaluation["concept"]["model_name"]}_{evaluation["concept"]["label"].replace(" ","_")}_{evaluation["created"]}.json'
    experiment_path = Path(experiment_filename)
    experiment_path.parent.mkdir(parents=True, exist_ok=True)
    json.dump(evaluation, open(experiment_filename, 'w'))

In [ ]:
candidates = classes_for_evaluation()

In [101]:
for cls in candidates[1:20]:
    time.sleep(60)
    print("Evaluating", cls["label"], "...")
    evaluate(cls)

Evaluating Autobiography ...
Evaluating Newspaper in New South Wales ...
Evaluating Player of American football from Alabama ...
Evaluating PlayStation VR game ...
